In [ ]:
#@title Copyright 2025 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title Install Dependencies

!pip install mapclassify xyzservices

from google.colab import auth
from google.cloud import bigquery


# import numpy as np
import pandas as pd
import geopandas as gpd

import folium
import mapclassify
import xyzservices

tiles =  xyzservices.TileProvider.from_qms("Google Satellite Hybrid")

### 1. Authenticate and Initialize EE and BQ


In [ ]:
# Change to a project where you have EE access and EE User Role as well as BigQuery User.
project_id = "your-project-id"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}


auth.authenticate_user()
client = bigquery.Client(project=project_id)

### 2. Create a GeoDataFrame of Forest Data Partnership Example Plots

This [data comes from the Forest Data Parternship](https://raw.githubusercontent.com/forestdatapartnership/whisp/main/tests/fixtures/geojson_example.geojson)

In [ ]:
query = '''FROM ee-demos.WHISP_DEMO.input_examples plot
|> EXTEND
  ST_RegionStats(geometry,
                'ee://earthengine_public_data/fdap/forest_persistence_2020',
                'value',
                include => 'value >= 0.5') AS forest_2020,
  ST_RegionStats(geometry, 'ee://earthengine_public_data/fdap/cocoa_2023',
                'probability', include => 'probability >= 0.7') AS cocoa_2023,
  ST_RegionStats(geometry, 'ee://earthengine_public_data/fdap/palm_2023',
                'probability', include => 'probability >= 0.7') AS palm_2023,
  ST_RegionStats(geometry, 'ee://earthengine_public_data/fdap/rubber_2023',
                'probability', include => 'probability >= 0.7') AS rubber_2023,
  ST_Area(geometry) AS plotArea,
|> EXTEND cocoa_2023.area + palm_2023.area + rubber_2023.area AS commodityArea2023
|> WHERE commodityArea2023 > 0
     AND commodityArea2023 > plotArea - forest_2020.area
|> SELECT plotid, ROUND(plotArea) area,
  ROUND(100 * forest_2020.area/plotArea) forestPct2020,
  ROUND(100 * commodityArea2023/plotArea) commodityPct2023,
  ROUND(100 * cocoa_2023.area/plotArea) cocoaPct2023,
  ROUND(100 * palm_2023.area/plotArea) palmPct2023,
  ROUND(100 * rubber_2023.area/plotArea) rubberPct2023, geometry;'''

# Use the BQ client and ST_RegionStats
plots = pd.read_gbq(query,project_id=project_id)

# Convert the DataFrame to a GeoDataFrame, using the geometry column
plots_gdf = gpd.GeoDataFrame(plots, geometry=gpd.GeoSeries.from_wkt(plots.geometry), crs='epsg:4326')


In [ ]:
import pandas as pd
# Identify plots with high forestPct2020 and commodityPct2023
high_deforestation_plots = plots_gdf[(plots_gdf['forestPct2020'] > 50) & (plots_gdf['commodityPct2023'] > 50)]

# Print the results
high_deforestation_plots

In [ ]:
# Find the plot with the highest commodityPct2023
max_commodity_plot = plots_gdf.loc[plots_gdf['commodityPct2023'].idxmax()]

# Get the coordinates of the highest value plot
# Access the centroid's x and y coordinates instead of the Polygon's x and y
max_latitude = max_commodity_plot.geometry.centroid.y
max_longitude = max_commodity_plot.geometry.centroid.x

# Instead of m.zoom_to, set location and zoom_start when creating the map
m = folium.Map(location=[max_latitude, max_longitude], zoom_start=17, tiles=tiles, attr="Google Maps")

# Add the GeoDataFrame layer to the map
plots_gdf.explore(
    "commodityPct2023",
    cmap="RdYlGn_r",
    scheme="NaturalBreaks",
    tooltip=["forestPct2020", "commodityPct2023"],
    m=m,  # Add the existing map object
    name="Likely Deforsted Plots"
)

# Display the map
m